In [1]:
# import dependencies 
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from config import gkey

In [2]:
# read csv file 
cleaned_file = "wine_fulldata.csv"

wine_df = pd.read_csv(cleaned_file)

In [3]:
# preview df
wine_df

,Unnamed: 0,id,country,description,designation,points,price,province,first_region,second_region,taster_name,taster_twitter_handle,title,variety,winery
0,4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
1,10,10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
2,23,23,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Matt Kettmann,@mattkettmann,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,Bianchi
3,25,25,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa
4,35,35,US,As with many of the Erath 2010 vineyard design...,Hyland,86,50.0,Oregon,McMinnville,Willamette Valley,Paul Gregutt,@paulgwine,Erath 2010 Hyland Pinot Noir (McMinnville),Pinot Noir,Erath
5,60,60,US,"Syrupy and dense, this wine is jammy in plum a...",Estate,86,100.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Okapi 2013 Estate Cabernet Sauvignon (Napa Val...,Cabernet Sauvignon,Okapi
6,62,62,US,"The aromas are brooding, with notes of barrel ...",Alder Ridge Vineyard,86,25.0,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Ram 2014 Alder Ridge Vineyard Cabernet Franc (...,Cabernet Franc,Ram
7,64,64,US,There are intriguing touches to the nose of th...,Golden Horn,86,26.0,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Sevtap 2015 Golden Horn Sauvignon Blanc (Santa...,Sauvignon Blanc,Sevtap
8,67,67,US,"A blend of Merlot and Cabernet Franc, this win...",Inspired,86,46.0,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Basel Cellars 2013 Inspired Red (Columbia Vall...,Bordeaux-style Red Blend,Basel Cellars
9,71,71,US,Big oak defines this robustly dense and extrac...,Old Vine,86,40.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Eco Terreno 2013 Old Vine Cabernet Sauvignon (...,Cabernet Sauvignon,Eco Terreno


In [9]:
#combine first region column and second region column to get a new column (for higher accuracy in google api pull for coords)
wine_df['region'] = wine_df['first_region'] + ' ' + wine_df['second_region']

In [10]:
unique_regions = wine_df['region'].unique()
wine_df['region'].nunique()

171

In [11]:
# extract region column and get all unique values
# unique_regions = wine_df['first_region'].unique()

# print(unique_regions)
# wine_df['first_region'].nunique()

In [12]:
# iterate through each unique region and do API pull to get their longitude and latitudes 
lat = []
lng = []
for region in unique_regions:
    query_url = 'https://maps.googleapis.com/maps/api/geocode/json?address=' + region + '&key=' + gkey
    response = requests.get(query_url).json()
    try:
        latitude = response['results'][0]['geometry']['location']['lat']
        longitude = response['results'][0]['geometry']['location']['lng']
    except:
        latitude = float('NaN')
        longitude = float('NaN')
        print(region + ': lat, lng not found')
    lat.append(latitude)
    lng.append(longitude)
#     wine_df['latitude'] = response['results'][0]['geometry']['location']['lat']
#     location = response['results'][0]['geometry']['location']
#     latitude = location['lat']
#     longitude = location['lng']

California California Other: lat, lng not found
Fair Play Sierra Foothills: lat, lng not found
Cienega Valley Central Coast: lat, lng not found


In [6]:
# create a list of dictionaries for each region
# coords_list = []
# for i in range(len(unique_regions)):
#     coords_dict = {}
#     coords_dict['region'] = unique_regions[i]
#     coords_dict['lat'] = lat[i]
#     coords_dict['lng'] = lng[i]
#     coords_list.append(coords_dict)

In [7]:
# pprint(coords_list)

In [13]:
# make a dictionary with region name as key and list of lat,lng as value
coords_dict = {}
for i in range(len(unique_regions)):
    coords_dict[unique_regions[i]] = [lat[i], lng[i]]


In [14]:
pprint(coords_dict)

{'Adelaida District Central Coast': [35.645236, -120.802497],
 'Alexander Valley Sonoma': [38.65649, -122.786907],
 'Alta Mesa Central Valley': [33.264274, -117.084515],
 'Amador County Sierra Foothills': [38.3488921, -120.7740931],
 'Ancient Lakes Columbia Valley': [47.1508517, -119.9631178],
 'Applegate Valley Southern Oregon': [42.276196, -123.219005],
 'Arroyo Grande Valley Central Coast': [35.1246535, -120.5691202],
 'Arroyo Seco Central Coast': [36.4135756, -121.3424305],
 'Atlas Peak Napa': [38.450743, -122.2627529],
 'Ballard Canyon Central Coast': [34.6376436, -120.1418336],
 'Ben Lomond Mountain Central Coast': [37.072154, -122.0838013],
 'Bennett Valley Sonoma': [38.4023607, -122.6404809],
 'Calaveras County Sierra Foothills': [38.1960477, -120.6805037],
 'California California Other': [nan, nan],
 'Calistoga Napa': [38.5787965, -122.5797054],
 'Capay Valley Central Valley': [38.5795434, -121.4852222],
 'Carmel Valley Central Coast': [36.4799024, -121.7327931],
 'Carneros Na

In [17]:
# iterate through each row of original df to find their respective coords
latitude2 = []
longitude2 = []
for index, row in wine_df.iterrows():
    if row['region'] in coords_dict:
        latitude2.append(coords_dict[row['region']][0])
        longitude2.append(coords_dict[row['region']][1])

In [18]:
#append coords to original df
wine_df['Latitude'] = latitude2
wine_df['Longitude'] = longitude2
wine_df.head()

,Unnamed: 0,id,country,description,designation,points,price,province,first_region,second_region,taster_name,taster_twitter_handle,title,variety,winery,region,Latitude,Longitude
0,4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Willamette Valley Willamette Valley,44.942554,-122.933762
1,10,10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,Napa Valley Napa,38.297538,-122.286865
2,23,23,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Matt Kettmann,@mattkettmann,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,Bianchi,Paso Robles Central Coast,35.613485,-120.684770
3,25,25,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa,Sonoma Coast Sonoma,38.577956,-122.988832
4,35,35,US,As with many of the Erath 2010 vineyard design...,Hyland,86,50.0,Oregon,McMinnville,Willamette Valley,Paul Gregutt,@paulgwine,Erath 2010 Hyland Pinot Noir (McMinnville),Pinot Noir,Erath,McMinnville Willamette Valley,45.210116,-123.198716


In [19]:
wine_df = wine_df.dropna(how='any')

In [20]:
wine_df = wine_df.drop(columns=['Unnamed: 0', 'id'])

In [21]:
#save as new csv
wine_df.to_csv('wine_data_coords_2.csv')

21768